## Item_based 추천 구현
* 동시 구매 count 
* 동시 구매 기반 조건부 확률

In [87]:
import pandas as pd
from collections import defaultdict

# 예제 데이터프레임
df_sample = pd.DataFrame({
    'user_id': [3, 3, 3, 3, 1, 1, 1, 2, 2, ],
    'product_id': [101, 102, 105, 106, 101, 102, 103, 101, 104],
    'event_type': ['cart'] * 9
})

# 'cart' 이벤트만 필터링
cart_df = df_sample[df_sample['event_type'] == 'cart']

In [49]:
user_products =cart_df.groupby('user_id')['product_id'].apply(list)

In [50]:
user_products

user_id
1         [101, 102, 103]
2              [101, 104]
3    [101, 102, 105, 106]
Name: product_id, dtype: object

In [51]:
## 동시구매에 대한 카운팅을 해야 한다.
## 101,102,103 담았으면 (101,102) (101,103) (102,103)

products_pairs = defaultdict(int)

for products in user_products:
    for i in range(len(products)):
        for j in range( i +1 , len(products)):
            pair =tuple(sorted([products[i], products[j]]))
            products_pairs[pair] +=1

In [52]:
products_pairs

defaultdict(int,
            {(101, 102): 2,
             (101, 103): 1,
             (102, 103): 1,
             (101, 104): 1,
             (101, 105): 1,
             (101, 106): 1,
             (102, 105): 1,
             (102, 106): 1,
             (105, 106): 1})

In [53]:
# itertools 를 활용한 조합 선정과 counting (시간 복잡도 면에서 좋음)
from itertools import combinations
from collections import Counter

all_combinations = []
for l in user_products:
    all_combinations +=list(combinations(sorted(l), 2))

pair_counts = dict(Counter(all_combinations))

pair_counts

{(101, 102): 2,
 (101, 103): 1,
 (102, 103): 1,
 (101, 104): 1,
 (101, 105): 1,
 (101, 106): 1,
 (102, 105): 1,
 (102, 106): 1,
 (105, 106): 1}

In [54]:
products_counts = cart_df['product_id'].value_counts()
products_counts

product_id
101    3
102    2
105    1
106    1
103    1
104    1
Name: count, dtype: int64

In [55]:
# P(prod2|prod1)
# P(prod1|prod2)

data = []

for (prod1, prod2), count in products_pairs.items():
    #P(prod2|prod1)
    if products_counts[prod1]>0:
        data.append([prod1, prod2, count / products_counts[prod1]])
    if products_counts[prod2]>0:
        data.append([prod2, prod1, count / products_counts[prod2]])

cond_prob_df =pd.DataFrame(data, columns = ['시작','이동','Confidence'])
        

In [56]:
cond_prob_df.sort_values(by='시작')

,시작,이동,Confidence
0,101,102,0.666667
10,101,106,0.333333
6,101,104,0.333333
8,101,105,0.333333
2,101,103,0.333333
4,102,103,0.500000
1,102,101,1.000000
12,102,105,0.500000
14,102,106,0.500000
5,103,102,1.000000


### 베이지안 확률을 통해 추천시스템을 구현하자!
-  베이지안 네트워크는 다른 변수들까지 추가해서 추천할 수도 있다.
    - 베이지안
        - 불확실성에 대한 믿음의 정도
        - 사전 확률 (사전지식) + 데이터가 새롭게 들어오면 -> 확률 갱신하게 된다.
        
     - 추천시스템에서는 어떤 식으로 이해하면 될까?
         - 일반적인 빈도 기반
             - 101과 102 , 101과 102가 같이 담긴 경우가 20번이네 -> 101는 전체가 40번 담겨있다 -> P(B|A) = 1/2 0.5
             - 관측된 카운팅 기반 의존
         - 베이지안 방식 추천
             - 101과 102가 20번 함께 담겼네 
                 - 102가 자주 담기는 제품이다 -> P(102) 높다.
                     - 그러면 101 함께 담긴 게 우연일 수도 있겠네.
             -> 단순한 빈도 이상의 불확실성에 대한 확률 해석을 추가하는 것
      - P(H|D) = P(D|H)* P(H) / P(D)
          - H : 사용자가 B를 담을 것이다. 좋아한다 가설의 개념
          - D : 실제 관측된 A를 담았다는 것 
          - P(H) : 기본적으로 유저가 B를 좋아할 (담을) 기본 확률
          - P(D|H) : 가능도(Likelihood) 유저가 B를 좋아한다면 A도 담았을 확률
          - P(H|D) : 사후확률(posterior) 유저가 A를 담은 걸 알고 있을 때 B를 좋아할(담을) 확률
      
      - 추천 시스템의 사후확률은?
          - 어떤 제품 B가 추천될 확률을 기존 데이터뿐만이 아니라 -> 제품에 대한 유저의 선호도 등의 정보를 반영해서 업데이트 한다.
          
--------
- 예를 들어서
    - 일반 조건부 확률 사용자가 제품 X를 담았을 때 -> 제품 Y도 담을 확률 얼마나 될까?
        - P(Y|X) -> count(X,Y) / count(X)]

- 베이지안 방식
    - P(Y|X) = P(X|Y) P(Y)/ P(X)
    - P(Y) Y가 원래 잘 팔리는지에 대한 정보 (사전정보)
    - P(X|Y) Y를 담은 사람들이 X도 담았은지? 연관도
    - P(Y|X) = 유저가 X를 담은 정보를 알 때 -> Y를 담을 확률 (사후확률)
        

- P(A|B) = P(AnB) / P(B)
- P(B) - 전체 장바구니에 대한 제품 B가 등장할 확률
- P(A) - 전체 장바구니에 대한 제품 A가 등장할 확률
- 베이지안 확률 P(B|A) 

In [57]:
# 예제 데이터프레임
df_sample = pd.DataFrame({
    'user_id': [1, 1, 1, 2, 2, 3, 3, 3, 3],
    'product_id': [101, 102, 103, 101, 104, 101, 102, 105, 106],
    'event_type': ['cart'] * 9
})

# 'cart' 이벤트만 필터링
cart_df = df_sample[df_sample['event_type'] == 'cart']

In [59]:
products_pairs = defaultdict(int)
user_products =cart_df.groupby('user_id')['product_id'].apply(list)

for products in user_products:
    for i in range(len(products)):
        for j in range( i +1 , len(products)):
            pair =tuple(sorted([products[i], products[j]]))
            products_pairs[pair] +=1

products_counts = cart_df['product_id'].value_counts()

In [67]:
products_counts[101]

3

In [60]:
# 전체 카운팅
total_carts= len(cart_df)

In [74]:
user_products

user_id
1         [101, 102, 103]
2              [101, 104]
3    [101, 102, 105, 106]
Name: product_id, dtype: object

In [73]:
products_pairs

defaultdict(int,
            {(101, 102): 2,
             (101, 103): 1,
             (102, 103): 1,
             (101, 104): 1,
             (101, 105): 1,
             (101, 106): 1,
             (102, 105): 1,
             (102, 106): 1,
             (105, 106): 1})

In [81]:
# 특정 제품 selected, prdoucts pairs, products_counts, total_carts, threshold = ?
def bayesian_recommendation(selected_product, products_pairs, products_counts , total_carts, threshold = 0.1):
    recommendation = {}
    for (prod1, prod2), count in products_pairs.items():
       
        if prod1 == selected_product or prod2 == selected_product:
            other_prod = prod2 if prod1 == selected_product else prod1
        
        #베이지안에 대한 코드가 필요하다.
        P_A_given_B = count / products_counts[other_prod]
        
        # P(B)
        P_B = products_counts[other_prod] / total_carts
        
        # P(A)
        P_A = products_counts[selected_product] / total_carts 
        
        # 베이지안 확률
        P_B_given_A = (P_A_given_B * P_B) / P_A
        print(P_A_given_B, P_B,P_A,P_B_given_A)
        if P_B_given_A >= threshold:
            recommendation[other_prod] = P_B_given_A
            
    return sorted(recommendation.items(), key = lambda x: x[1], reverse=True)

In [88]:
# 베이즈 정리 기반 추천 함수
def bayesian_recommendation(selected_product, products_pairs, products_counts, total_carts, threshold=0.1):
    recommendation = {}
    for (prod1, prod2), count in products_pairs.items():
        # selected_product가 포함된 쌍만 처리
        if selected_product not in (prod1, prod2):
            continue
        
        other_prod = prod2 if prod1 == selected_product else prod1

        # P(A|B)
        P_A_given_B = count / products_counts[other_prod]

        # P(B)
        P_B = products_counts[other_prod] / total_carts

        # P(A)
        P_A = products_counts[selected_product] / total_carts

        # P(B|A) = (P(A|B) * P(B)) / P(A)
        P_B_given_A = (P_A_given_B * P_B) / P_A

        if P_B_given_A >= threshold:
            recommendation[other_prod] = P_B_given_A

    return sorted(recommendation.items(), key=lambda x: x[1], reverse=True)

In [86]:
selected_product = 102
recommendation = {}
for (prod1, prod2), count in products_pairs.items():
    
    if prod1 == selected_product or prod2 == selected_product:
        other_prod = prod2 if prod1 == selected_product else prod1
    
    #베이지안에 대한 코드가 필요하다.
    P_A_given_B = count / products_counts[other_prod]
    
    # P(B)
    P_B = products_counts[other_prod] / total_carts
    
    # P(A)
    P_A = products_counts[selected_product] / total_carts 
    
    # 베이지안 확률
    P_B_given_A = (P_A_given_B * P_B) / P_A
    
    recommendation[other_prod] = P_B_given_A

recommendation

{101: 0.5, 103: 0.5, 105: 0.5, 106: 0.5}

In [89]:
selected_product=102
recommend_products=bayesian_recommendation(selected_product, products_pairs, products_counts,total_carts)

In [64]:
selected_product=101
recommend_products=bayesian_recommendation(selected_product, products_pairs, products_counts,total_carts)
recommend_products

[(102, 0.6666666666666666),
 (103, 0.3333333333333333),
 (104, 0.3333333333333333),
 (105, 0.3333333333333333),
 (106, 0.3333333333333333)]